In [1]:
import torch
# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")

if cuda_available:
    # Get the number of GPUs available
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")

    # Get the name of each GPU
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")

    # Get the current GPU memory usage
    for i in range(num_gpus):
        gpu_memory_allocated = torch.cuda.memory_allocated(i)
        gpu_memory_reserved = torch.cuda.memory_reserved(i)
        print(f"GPU {i} memory allocated: {gpu_memory_allocated / (1024 ** 3):.2f} GB")
        print(f"GPU {i} memory reserved: {gpu_memory_reserved / (1024 ** 3):.2f} GB")
else:
    print("CUDA is not available.")

CUDA available: True
Number of GPUs available: 1
GPU 0: NVIDIA GeForce RTX 4060 Laptop GPU
GPU 0 memory allocated: 0.00 GB
GPU 0 memory reserved: 0.00 GB


In [ ]:
import sys
import os
import pickle
import logging
import numba
import matplotlib
import matplotlib.pyplot as plt
logging.getLogger('matplotlib.font_manager').disabled = True
numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

matplotlib_logger = logging.getLogger('matplotlib')
matplotlib_logger.setLevel(logging.WARNING)

# Add the src directory to sys.path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
print(src_path)
sys.path.append(src_path)

from src.IndependentSteps import Pycromanager2NativeDataType, FFF2NativeDataType, SingleTIFF2NativeDataType

from src.SequentialSteps import BIGFISH_SpotDetection, SimpleCellposeSegmentaion, IlluminationCorrection, \
                                Calculate_BIGFISH_Threshold, CellProperties, Automatic_BIGFISH_Threshold

from src.FinalizationSteps import Save_Outputs, Save_Images, Save_Parameters, Save_Masks, return_to_NAS, remove_local_data_but_keep_h5, \
                                     remove_local_data

from src.Parameters import Parameters, Experiment, Settings, ScopeClass, DataContainer

from src.Displays import Display

from src.GUI import GUI, StepGUI

from src.Pipeline import Pipeline

c:\Users\Jack\Documents\GitHub\AngelFISH


INFO:numexpr.utils:Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO:numexpr.utils:NumExpr defaulting to 16 threads.
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7


In [3]:
scope = ScopeClass() 
data = DataContainer(clear_after_error=False)
settings = Settings(name='demo')
experiment = Experiment()

In [4]:
# you can change the parameters
experiment.initial_data_location = r'Users/Jack/HistoneModification_ScratchAssay/Full 24 hour scratch assay.tif'

experiment.nucChannel = 0
experiment.cytoChannel = 0
experiment.voxel_size_z = 500

settings.num_chunks_to_run = 5 # this is the limit to how many chunks you want to run and it will stop after that
settings.sigma_dict = {0: 50, 1: 100, 2: 100}

Parameters().get_parameters()

{'voxel_size_yx': 130,
 'spot_z': 500,
 'spot_yx': 360,
 'microscope_saving_format': 'pycromanager',
 'state': 'global',
 'local_dataset_location': None,
 'h5_file': None,
 'total_num_chunks': None,
 'images': None,
 'masks': None,
 'temp': None,
 'clear_after_error': False,
 'name': 'demo',
 'return_data_to_NAS': True,
 'NUMBER_OF_CORES': 4,
 'save_files': True,
 'num_chunks_to_run': 5,
 'download_data_from_NAS': True,
 'connection_config_location': 'c:\\Users\\Jack\\Documents\\GitHub\\AngelFISH\\config_nas.yml',
 'share_name': 'share',
 'display_plots': True,
 'load_in_mask': False,
 'mask_structure': {'masks': ('ptczyx', None, None),
  'cell_mask': ('zyx', 'cytoChannel', 'masks'),
  'nuc_mask': ('zyx', 'nucChannel', 'masks')},
 'order': 'pt',
 'sigma_dict': {0: 50, 1: 100, 2: 100},
 'initial_data_location': 'Users/Jack/HistoneModification_ScratchAssay/Full 24 hour scratch assay.tif',
 'index_dict': None,
 'nucChannel': 0,
 'cytoChannel': 0,
 'FISHChannel': None,
 'voxel_size_z': 500

In [5]:
SingleTIFF2NativeDataType().run()

INFO:SMB.SMBConnection:Authentication with remote machine "MUNSKY-NAS.ENGR.COLOSTATE.EDU" for user "formanj" will be using NTLM v2 authentication (with extended security)
INFO:SMB.SMBConnection:Now switching over to SMB2 protocol communication
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_NEGOTIATE" (command:0x0000 flags:0x0001)
INFO:SMB.SMBConnection:SMB2 dialect negotiation successful
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_SESSION_SETUP" (command:0x0001 flags:0x0001)
INFO:SMB.SMBConnection:Performing NTLMv2 authentication (on SMB2) with server challenge "b'c52b74769484268a'"
INFO:SMB.SMBConnection:Performing NTLMv2 authentication (on SMB2) with server challenge "b'c52b74769484268a'"
DEBUG:SMB.SMBConnection:NT challenge response is "b'756803e246742f94b01f463feea0f697010100000000000000000000000000003329bed767278ae7000000000200100045004e00470052005f0044004f004d00010014004d0055004e0053004b0059002d004e00410053000400240065006e00670072002e0063006f006c006f007300740

Connection established


DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_READ" (command:0x0008 flags:0x0003)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_READ" (command:0x0008 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_READ" (command:0x0008 flags:0x0003)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_READ" (command:0x0008 flags:0x0003)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_READ" (command:0x0008 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_READ" (command:0x0008 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_READ" (command:0x0008 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_READ" (command:0x0008 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_READ" (command:0x0008 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_READ" (command:0x0008 flags:0x0001)
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_READ" (command:0x0008 fl

Files downloaded to: c:\Users\Jack\Documents\GitHub\AngelFISH\dataBases\Full 24 hour scratch assay.tif


IndexError: list index out of range

In [ ]:
Display().displayImage_maxProject(channel=2)

In [ ]:
settings.cellpose_min_size = 500
settings.cellpose_diameter = [180, 90]
settings.cellpose_pretrained_model = ["GAPDH_cyto", 'DAPI_nuclei'] 

SimpleCellposeSegmentaion().run()

In [ ]:
CellProperties().run()